In [24]:
from itertools import product
from functools import reduce
from random import random, randint, seed, choice
import numpy as np
from scipy import sparse
from copy import copy
import math

In [18]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    # Crea una matrice sparsa inizializzata a false
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)

    # Itera su tutti i possibili indici di riga e colonna della matrice, usando la funzione product del modulo itertools. 
    # Per ogni coppia di indici (s, p), assegna il valore True alla matrice con probabilità density. 
    # Questo significa che l’insieme s copre l’elemento p con probabilità density.
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True

    # Questo pezzo di codice mi assicura che la matrice creata generi almeno una soluzione di covering.
    # almeno un set copre ogni elemento dell’insieme universale, garantendo che esista una soluzione al problema 
    # di copertura di insiemi.
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge
Find the best solution with the fewest calls to the fitness functions for:

- num_points = [100, 1_000, 5_000]
- num_sets = num_points
- density = [.3, .7]

In [61]:
# Parameters of the problem

NUM_POINTS = 100
NUM_SETS = NUM_POINTS
DENSITY = 0.7

# This parameter is useful to decide the probability of taking a set in the initial state
# since the goal is to find the covering set with the less tiles possible it seems better to start 
# from an initial state with very few tiles taken (0 if possible)
STARTING_PROB = 0.001

SETS = make_set_covering_problem(NUM_POINTS, NUM_SETS, DENSITY)

# Counter of the call to the fitness function
call_count = 0

In [22]:
def fitness(state):
    # I update the number of call to the fitness function
    global call_count
    call_count += 1
    
    cost = sum(state) # How many sets I am using
    covered_elements = np.sum(
        reduce(
            np.logical_or,
            [SETS[[i]].todense() for i, t in enumerate(state) if t],
            np.array([False for _ in range(NUM_POINTS)])
        )
    )
    return covered_elements, -cost


# Tweak function
def tweak(state):
    new_state = copy(state)
    index = randint(0, NUM_SETS - 1)
    new_state[index] = not new_state[index]
    return new_state

In [62]:
# Generating the first random state (chosing a set with a certain probability)
starting_probability = [True] + [False for _ in range(int(1/STARTING_PROB)-1)]
initial_state = [choice(starting_probability) for _ in range(NUM_SETS)]

### Steepest ascent hill-climbing

In [31]:
def steepest_ascent_hill_climbing(current_state, num_steps, dim_neighborhood):
    print('STARTING steepest ascent hill climbing...')
    global call_count
    call_count = 0

    best_fitness = fitness(current_state)
    void_steps = 0
    print(f'Initial state: {best_fitness}\nNumber of steps: {num_steps}')

    for step in range(num_steps):
        state_r = tweak(current_state) # I change one set (from taken to not_taken and viceversa)
        fitness_r = fitness(state_r)

        # Then I search if in the neighborhood there's something better
        # The idea is: if I find a better state I change another set, so
        # potentially I could change dim_neighborhood sets (from taken to not taken and viceversa)
        for _ in range(dim_neighborhood): 
            state_w = tweak(state_r)
            fitness_w = fitness(state_w)
            if fitness_w > fitness_r:
                state_r = state_w
                fitness_r = fitness_w

        # If the state result from the local search is better than the one from which i started (current_state)
        # I update it 
        if fitness_r > best_fitness:
            current_state = state_r
            best_fitness = fitness_r
            print(f'At step {step+1} I have improved the solution: {best_fitness}')
            void_steps = 0
        else:
            void_steps += 1
            # If i passed the last 10% of step without any improvement I stop
            if void_steps >= 0.1*num_steps:
                break
    
    return best_fitness

In [71]:
result = steepest_ascent_hill_climbing(initial_state, 1_000, 5)
print(f'STEEPEST HILL CLIMBING ----> {result}, with {call_count} calls to the fitness function')

# RISULTATI (Da inserire poi nel report)
# NUM_SETS = NUM_POINTS = 5000
# DENSITY = .3 e STARTING_PROB = 0.001
# num_steps = 10_000
# dim_neighborhood = 5
# best -----> (5000, -22) with 20917 calls to the fitness function

# ----------------------------------------------------------------------
# NUM_SETS = NUM_POINTS = 5000
# DENSITY = .7 e STARTING_PROB = 0.001
# num_steps = 10_000
# dim_neighborhood = 5
# best -----> (5000, -7) with 6007 calls to the fitness function

# ----------------------------------------------------------------------
# NUM_SETS = NUM_POINTS = 1000
# DENSITY = .3 e STARTING_PROB = 0.001
# num_steps = 10_000
# dim_neighborhood = 5
# best -----> (1000, -14) with 12247 calls to the fitness function

# ----------------------------------------------------------------------
# NUM_SETS = NUM_POINTS = 1000
# DENSITY = .7 e STARTING_PROB = 0.001
# num_steps = 10_000
# dim_neighborhood = 5
# best -----> (1000, -5) with 8239 calls to the fitness function

# ----------------------------------------------------------------------
# NUM_SETS = NUM_POINTS = 100
# DENSITY = .3 e STARTING_PROB = 0.001
# num_steps = 1_000
# dim_neighborhood = 5
# best -----> (100, -7) with 715 calls to the fitness function

# ----------------------------------------------------------------------
# NUM_SETS = NUM_POINTS = 100
# DENSITY = .7 e STARTING_PROB = 0.001
# num_steps = 1_000
# dim_neighborhood = 5
# best -----> (100, -3) with 649 calls to the fitness function


STARTING steepest ascent hill climbing...
Initial state: (0, 0)
Number of steps: 1000
At step 1 I have improved the solution: (100, -4)
At step 57 I have improved the solution: (100, -3)
STEEPEST HILL CLIMBING ----> (100, -3), with 943 calls to the fitness function


### Simulated annealing


In [86]:
# Function that computes the difference of two evaluation of two different states
# Easy idea: difference of the covered elements + difference of the number of sets
def diff_fitness(fitA, fitB):
    result = (fitA[0] - fitB[0]) + (fitA[1]-fitB[1])
    return result

In [149]:
def simulated_annealing(current_state, num_steps, t): # t is the initial temperature
    print('STARTING simulated annealing...')
    global call_count
    call_count = 0
    print(f'Initial state: {fitness(current_state)}\nNumber of steps: {num_steps}, Initali Temperature: {t}')

    best_fitness = fitness(current_state)
    curr_fitness = best_fitness
    for step in range(num_steps):
        candidate = tweak(current_state)
        candidate_fitness = fitness(candidate)
        if candidate_fitness > best_fitness:
            best_state = candidate
            best_fitness = candidate_fitness
            print(f'At step {step+1} I have improved the solution: {best_fitness}')
            void_steps = 0
        diff = diff_fitness(curr_fitness, candidate_fitness)
        temp = t / float(step+1)
        prob = math.exp(-diff / temp)
        if diff < 0 or random() < prob:
            current_state = candidate
            curr_fitness = candidate_fitness

    return best_fitness

In [150]:
result = simulated_annealing(initial_state, 10_000, t=100)
print(f'SIMULATED ANNEALING ----> {result}, with {call_count} calls to the fitness function')

STARTING simulated annealing...
Initial state: (5000, -41)
Number of steps: 10000, Initali Temperature: 100
At step 6718 I have improved the solution: (5000, -40)
At step 6721 I have improved the solution: (5000, -39)
At step 6758 I have improved the solution: (5000, -38)
At step 6822 I have improved the solution: (5000, -37)
At step 6886 I have improved the solution: (5000, -36)
At step 7031 I have improved the solution: (5000, -35)
At step 7055 I have improved the solution: (5000, -34)
At step 7264 I have improved the solution: (5000, -33)
At step 7351 I have improved the solution: (5000, -32)
At step 7532 I have improved the solution: (5000, -31)
At step 7853 I have improved the solution: (5000, -30)
At step 8176 I have improved the solution: (5000, -29)
At step 8178 I have improved the solution: (5000, -28)
At step 8384 I have improved the solution: (5000, -27)
At step 9285 I have improved the solution: (5000, -26)
At step 9453 I have improved the solution: (5000, -25)
At step 9515

In [17]:
p = [True, False, False, False]
q = [False, False, False, False]
z = [True, True, True, False]

vect = [q,z]
print(vect)

distances = []
for i in range(2):
    distances.append(math.dist(p, vect[i]))

print(distances)
print(math.dist([1, 0, 0, 0],[1, 1, 1, 0]))
# Prendo il numero di True non presenti in state e ne faccio la radice

[[False, False, False, False], [True, True, True, False]]
[1.0, 1.4142135623730951]
1.4142135623730951
